In [ ]:
import os

try:
    has_changed_dir
except:
    has_changed_dir = False

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install datasets
    %pip install translate-toolkit
    #%pip install bitsandbytes

    !git clone https://github.com/MartinKirkegaardDK/KDS_MI.git

    if not has_changed_dir:
        os.chdir('KDS_MI')
        has_changed_dir = True
else:
    if not has_changed_dir:
        os.chdir('.')
        has_changed_dir = True

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.probe_confidence_intervals import model_setup
from utils.steering import generate_with_steering, loss_with_steering
from classes.datahandling import ParallelDataset
import torch

In [ ]:
model_name  = "EleutherAI/pythia-14m"
model_name = "AI-Sweden-Models/gpt-sw3-356m"

model, tokenizer, device = model_setup(model_name)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [ ]:
def split_after_delim(string, delim):
    try: 
        first, second = string.split(delim, maxsplit=1)
    except ValueError:
        return string, ''

    return first + delim, second

In [ ]:
ds = ParallelDataset.from_tmx('data/small-da-en.tmx', 'da', 'en')

In [ ]:
da_prompt, da_continuation = split_after_delim(ds[3]['da'], ',')
en_prompt, en_continuation = split_after_delim(ds[3]['en'], ',')

In [ ]:
steering_vector = torch.load('steering_vectors\initial_test_run\combined_steering_vector_layer_15_tensor.pt')

In [ ]:
generate_with_steering(
    model,
    tokenizer,
    15,
    [en_prompt],
    steering_vector,
    10)